# **Create a Collateralized Debt Position**

#### Creating a Collateralized Debt Position (CDP) within the Halalend Protocol on the Cardano blockchain involves a series of steps and meeting certain criteria to ensure the transaction's validity and compliance with the protocol's rules. The process can be outlined and simplified as follows:

## Qualification Criteria for a CDP:

- **Loan Amount and Collateral Ratio:** The amount you wish to borrow must meet or exceed the protocol's minimum collateral ratio. This ensures that the loan is overcollateralized, providing a buffer against market volatility.
- **Allowed Collateral Assets:** Only specific assets approved by the Halalend Protocol can be used as collateral. These assets are listed in the protocol parameters, which may be subject to updates.
- **Liquidity Requirements:** The liquidity pool must have sufficient funds to cover the loan you're requesting.
- **Fees:** Borrowers are required to pay a protocol usage fee, the ADA fee set by the Cardano Network, and optionally, a batcher fee for expedited processing.

## Key Protocol Parameters:

- **Minimum Collateral Ratio:** Determines the required overcollateralization for loan approval.
- **Minimum Loan Amount:** Sets the threshold for the smallest loan amount permissible.
- **Protocol Usage Fee:** A fee charged for using the Halalend Protocol to create a loan position.
- **Loan Term:** Specifies how long the loan will last before it needs to be repaid.
- **Collateral Assets**: A list of assets that can be used as collateral.

## On-chain Validation:

- **Liquidity Pool UTXO:** Represents the available liquidity for a given asset, critical for ensuring the loan can be supported.
- **Lending Validator:** A smart contract on Cardano that validates borrow requests and ensures they comply with protocol rules.
- **Protocol Parameters UTXO:** Stores the protocol's operational parameters and lending criteria for reference in transactions.
- **Loan Position NFT Minting Validator:** A smart contract responsible for creating NFTs that represent loan positions, allowing them to be traded or managed on secondary markets.
- **Oracle/DEX Liquidity Pool UTXO:** Provides current exchange rates for collateral and loan assets, essential for accurate valuation and risk assessment.


## Off-chain Infrastructure:

- **Batcher:** An essential background service that batches transactions for processing. This component is crucial due to Cardano's UTXO model, where a UTXO can only be consumed by one transaction in a block. The batcher ensures multiple transactions can be processed efficiently.

### Assuming the following Protocol Parameters:

In [16]:
let MINIMUM_COLLATERAL_RATIO = 1.5
let MINIMUM_LOAN_AMOUNT = 100.0
let PROTOCOL_USAGE_FEE = 5.0
let COLLATERAL_ASSETS = ["ada"; "iUSD"; "halalend"; "hosky"]
let LOAN_TERM = 14 * 24 * 60 * 60 * 1000   // 14 days in milliseconds

### And the following calculations:

##### *Note: these are simplified calculations*

In [17]:
open System.Collections.Generic

// Calculates the current collateral ratio of a loan position given the exchange rates of the collateral and the loan
let calc_collateral_ratio total_collateral_value_in_usd total_loan_amount_in_usd = total_collateral_value_in_usd / total_loan_amount_in_usd

// Checks if the collateral_assets are allowed
let only_allowed_assets collateral_assets = not (List.exists (fun asset -> not (List.contains asset COLLATERAL_ASSETS)) collateral_assets)

// Function to calculate total value for a single asset
let calc_collateral_asset_value (asset: IDictionary<string, obj>) =
    let exchangeRate = unbox<float>(asset.["exchange_rate"])
    let amount = unbox<float>(asset.["amount"])
    exchangeRate * amount

// Checks if the borrower is eligible for a loan
let is_eligible_for_loan assets collateral_usd loan_usd loan_term liquidity_usd =
    let collateral_ratio_met = calc_collateral_ratio collateral_usd loan_usd >= MINIMUM_COLLATERAL_RATIO
    let loan_amount_met = loan_usd >= MINIMUM_LOAN_AMOUNT
    let liquidity_met = liquidity_usd >= loan_usd
    let assets_met = only_allowed_assets assets
    let term_met = loan_term <= LOAN_TERM

    printf "CDP Qualification Results:\n"
    printf "Collateral ratio met: %b\n" collateral_ratio_met
    printf "Loan amount met: %b\n" loan_amount_met
    printf "Liquidity met: %b\n" liquidity_met
    printf "Assets met: %b\n" assets_met
    printf "Term met: %b\n\n" term_met

    collateral_ratio_met && loan_amount_met && liquidity_met && assets_met && term_met
    

### Now we need to define the loan parameters and current exchange rates and check for eligibility to create a CDP

In [18]:
// These are the collaterals sent by the borrower to the contract
let collateral_assets = 
    [("iUSD", dict [("exchange_rate", box 1.0); ("amount", box 1000.0)])
     ("halalend", dict [("exchange_rate", box 0.4); ("amount", box 1000.0)])
     ("hosky", dict [("exchange_rate", box 0.001); ("amount", box 100000.0)])]
    |> Map.ofList

let loan_term = 14 * 24 * 60 * 60 * 1000   // 14 days in milliseconds
let collateral_asset_list = collateral_assets |> Map.toList |> List.map fst
let total_collateral_value_usd = 
    collateral_assets
    |> Map.map (fun _ asset -> calc_collateral_asset_value asset)  // Calculate total for each asset
    |> Map.fold (fun acc _ value -> acc + value) 0.0       // Sum up all totals
let loan_usd = 1000.0 // this is the amount of the loan in USD
let liquidity_usd = 10000.0 // this is the converted value of the liquidity pool UTXO in USD

printf "Total collateral value: %f\n" total_collateral_value_usd
printf "Loan amount: %f\n" loan_usd
printf "Liquidity: %f\n\n" liquidity_usd

let can_loan = is_eligible_for_loan collateral_asset_list total_collateral_value_usd loan_usd loan_term liquidity_usd

if can_loan then
    printf "Loan approved\n"
    printf "Protocol usage fee: %f\n" PROTOCOL_USAGE_FEE
    printf "Remaining liquidity after loan: %f\n" (liquidity_usd - loan_usd )
    printf "Total locked collateral: %f\n" total_collateral_value_usd
else
    printf "Loan denied\n"


Total collateral value: 1500.000000
Loan amount: 1000.000000
Liquidity: 10000.000000

CDP Qualification Results:
Collateral ratio met: true
Loan amount met: true
Liquidity met: true
Assets met: true
Term met: true

Loan approved
Protocol usage fee: 5.000000
Remaining liquidity after loan: 9000.000000
Total locked collateral: 1500.000000
